# SVM-Klassifikation der Heise-Newsticker-Meldungen

Kontakt: christian.winkler@datanizing.com

In [ ]:
!test -f newsticker-2022-2023-good-bad.csv.gz || wget https://github.com/datanizing/m3-llm-workshop/raw/main/newsticker-2022-2023-good-bad.csv.gz

In [ ]:
import pandas as pd

df = pd.read_csv("newsticker-2022-2023-good-bad.csv.gz", index_col="id")

In [ ]:
# Labels auf Integer wandeln
df["label"] = 0
df.loc[df["quality"] == "good", "label"] = 1
df.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.de.stop_words import STOP_WORDS as stop_words
tfidf_vectorizer = TfidfVectorizer(stop_words=list(stop_words), min_df=2, ngram_range=(1,2))
tfidf_vectors = tfidf_vectorizer.fit_transform(df["headline"])
tfidf_vectors

In [ ]:
from sklearn.model_selection import train_test_split
(X_train, X_test, y_train, y_test) = train_test_split(tfidf_vectors, df["label"].values, 
                                                      train_size=0.75, random_state=42,
                                                      stratify=df["label"].values)

In [ ]:
from sklearn.svm import SVC
svm = SVC()
svc = svm.fit(X_train, y_train)

In [ ]:
pred_test  = svc.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred_test)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(10,8))
sns.heatmap(confusion_matrix(y_test, pred_test), annot=True)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred_test))